In [7]:
import Random
using Distributions
using Statistics
using DelimitedFiles
using Dates

LoadError: ArgumentError: Package Filesystem not found in current path:
- Run `import Pkg; Pkg.add("Filesystem")` to install the Filesystem package.


In [8]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},outcomes,storage,numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG,ntherm)
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,transcription_translation_rates!,storage,updatestorage!,rng,numsteps,maxsteps,ntherm)
    return storage,endstate,steps,weightsum
end

function reach_balance_test(initialstate,params,outcomes,numsteps,maxsteps,rng,flux_thres,cov_thres,tries,ntherm)
    statevector = initialstate
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    calcstor = BalanceEqData_nofb()
    fluxcheck = [0,0]
    cbvals = [0,0]
    
    while (trynum < tries) && (!thresholdmet)
        calcstor,statevector,steps,lastsimtime = data_collection(statevector,params,outcomes,calcstor,numsteps,maxsteps,rng,ntherm)
        weightsum = calcstor.weightsum
        fluxcheck = [relative_error(calcstor.meanRpX,calcstor.meanRmX),relative_error(calcstor.meanRpY,calcstor.meanRmY)]
        if (fluxcheck[1] < flux_thres) &&  (fluxcheck[2] < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum) 
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum) 
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum) 
            cbvals = [cbxx_comp,cbyy_comp,cbyx_comp]
            thresholdmet = all(cbvals .< cov_thres)               
        end
        trynum += 1
        totalsteps += steps
        if maxsteps>100000
            maxsteps = maxsteps ÷ 10
        end
    end
return calcstor,totalsteps,trynum,fluxcheck,cbvals
end

reach_balance_test (generic function with 1 method)

Run some number of simulations, storing the used parameters+simulation constraints. Save each simulation's output into a big CSV. Finally, go through the results and calculate the perturbation data

In [4]:
is = [10,10]
params = [50.,1.,50.,5.]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e6)
rng = Random.MersenneTwister()
fluxt = 0.01
covt = 0.01
tries = 20
ntherm = Int(10e3)
reach_balance_test(is,params,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)

(BalanceEqData_nofb(1960.427528278479, 49.9455226811168, 499.62484949817537, 97464.26173543432, 9.018199935329095e6, 807821.7991980104, 50.0, 49.94527825282817, 2497.263912641121, 2498.125414750803, 97446.75842761256, 0.0, 808001.5696178481, 0.0, 4.038329870038797e6, 4.87233792137949e6, 4.508635087504509e7, 4.040007848089755e7), 10000001, 1, [0.0010956340436192086, 0.0003448594312339664], [0.005346293554985924, 0.004663872274317854, 0.005366220211444509])

In [15]:
folder = "no_fb_sims_largerange3_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 30
is = [10,10]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e3)

paramsets = (
[50.,10.,5.,1.],
[40.,1.,2.,3.],
[10.,1.,20.,1.],
[40.,5.,40.,1.],
[10.,1.,15.,15.],
[25.,1.,2.,1.,],
[10.,1.,15.,5.],
[10.,0.1,5.,1.])
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for j = 1:nsims
        pertrng = Random.Xoshiro()
        statevector = copy(is)
        pertsize = rand(pertrng,Uniform(0.8,1.2))
        pertparams[j%4+1] *= pertsize
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,j%4+1,pertsize,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams[j%4+1] /= pertsize
    end
end


Paramset ([2.0, 1.0, 50.0, 10.0], [50.0, 10.0, 5.0, 1.0], [40.0, 1.0, 2.0, 3.0], [10.0, 1.0, 20.0, 1.0], [40.0, 5.0, 40.0, 1.0], [10.0, 1.0, 15.0, 15.0], [25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 15.0, 5.0])
Paramset ([2.0, 1.0, 50.0, 10.0], [50.0, 10.0, 5.0, 1.0], [40.0, 1.0, 2.0, 3.0], [10.0, 1.0, 20.0, 1.0], [40.0, 5.0, 40.0, 1.0], [10.0, 1.0, 15.0, 15.0], [25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 15.0, 5.0])
Paramset ([2.0, 1.0, 50.0, 10.0], [50.0, 10.0, 5.0, 1.0], [40.0, 1.0, 2.0, 3.0], [10.0, 1.0, 20.0, 1.0], [40.0, 5.0, 40.0, 1.0], [10.0, 1.0, 15.0, 15.0], [25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 15.0, 5.0])
Paramset ([2.0, 1.0, 50.0, 10.0], [50.0, 10.0, 5.0, 1.0], [40.0, 1.0, 2.0, 3.0], [10.0, 1.0, 20.0, 1.0], [40.0, 5.0, 40.0, 1.0], [10.0, 1.0, 15.0, 15.0], [25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 15.0, 5.0])
Paramset ([2.0, 1.0, 50.0, 10.0], [50.0, 10.0, 5.0, 1.0], [40.0, 1.0, 2.0, 3.0], [10.0, 1.0, 20.0, 1.0], [40.0, 5.0, 40.0, 1.0], [10.0, 1.0, 15.0, 15.0], [25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 15.0, 5

1×3 adjoint(::Vector{Int64}) with eltype Int64:
 1  2  3